# **A full 3D analysis using the low-level Gammapy API**

**Objective: Performing a full 3D anaysis of the extended source [MSH 15-52](http://tevcat.uchicago.edu/?mode=1;id=95)**

In practice, we have to:
- Prepare the **data access and selection**
  - Create a `~gammapy.data.DataStore` poiting to the relevant data 
  - Apply an observation selection to produce a list of observations, a `~gammapy.data.Observations` object.
- Set up the **analyis parameters**
  - Define a geometry of the Map we want to produce, with a sky projection and an energy range.
    - Create a `~gammapy.maps.MapAxis` for the energy
    - Create a `~gammapy.maps.WcsGeom` for the geometry
    - Define the exclusion mask
    - Choose the correct ~gammapy.datasets.Dataset type and define it
- Do the **data reduction**
  - Create the necessary makers : 
    - the map dataset maker : `~gammapy.makers.MapDatasetMaker`
    - the [background normalization](https://docs.gammapy.org/1.1/user-guide/makers/fov.html) maker, here a `~gammapy.makers.FoVBackgroundMaker`
    - and usually the safe range maker : `~gammapy.makers.SafeMaskMaker`
  - Perform the data reduction loop. And for every observation:
    - Apply the makers sequentially to produce the current `~gammapy.datasets.MapDataset`
    - Stack it on the target one.
- Make the **modeling and fitting**
  - Define the `~gammapy.modeling.models.SkyModel` to apply to the dataset.
  - Create a `~gammapy.modeling.Fit` object and run it to fit the model parameters
  - Apply a `~gammapy.estimators.FluxPointsEstimator` to compute flux points for the spectral part of the fit.

As support for this exercice, please refer to the [Low Level API tutorial](https://docs.gammapy.org/1.1/tutorials/starting/analysis_2.html).

## Setup
First, we setup the analysis by performing required imports.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
import numpy as np
import logging
from astropy import units as u
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion
from scipy.stats import norm

In [ ]:
from gammapy.data import DataStore
from gammapy.datasets import MapDataset
from gammapy.maps import WcsGeom, MapAxis
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import (
    SkyModel,
    PowerLawSpectralModel,
    PointSpatialModel,
    FoVBackgroundModel,
    GaussianSpatialModel,
    Models
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator, ExcessMapEstimator

## Optional set-up

In [ ]:
logging.basicConfig()    
log = logging.getLogger("1Danalysis")
log.setLevel(logging.WARNING) #INFO, WARNING, DEBUG

from astropy.io.fits.verify import VerifyWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', VerifyWarning)

from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True

## Defining the datastore and selecting observations

We first use the `~gammapy.data.DataStore` object to access the observations we want to analyse, here the H.E.S.S. DL3 DR1. 

In [ ]:
data_store = 

In [ ]:
pos = SkyCoord(228.32083333, -59.08166667, unit=u.deg, frame="icrs")
pos

We can now define an observation filter to select only the relevant observations. 
Here we use a cone search which we define with a python dict.

We then filter the `ObservationTable` with `~gammapy.data.ObservationTable.select_observations()`.

In [ ]:
obs_table_filtered = 
obs_ids = 

We can now retrieve the relevant observations by passing their `obs_id` to the`~gammapy.data.DataStore.get_observations()` method.

In [ ]:
observations = 

In [ ]:
# print(observations)

## Preparing reduced datasets geometry

Now we define a reference geometry for our analysis, We choose a WCS based geometry with a binsize of 0.02 deg and also define an energy axis: 

In [ ]:
energy_axis = 
geom = 
# Reduced IRFs are defined in true energy (i.e. not measured energy).
energy_axis_true = 

Now we can define the target dataset with this geometry.

In [ ]:
stacked = 

## Data reduction

### Create the maker classes to be used

We first initialize the `Maker` objects that will take care of the data reduction.

In [ ]:
maker = 
maker_safe_mask = 

In [ ]:
exclusion_mask =
maker_fov = 

In [ ]:
exclusion_mask.plot_interactive()

### Perform the data reduction loop

In [ ]:
%%time

for obs in observations:


In [ ]:
print(stacked)

### Inspect the reduced dataset

In [ ]:
stacked.counts.

In [ ]:
dataset.mask_safe.

In [ ]:
stacked.plot_residuals

## Compute an excess and a significance map

In [ ]:
estimator = 
lima_maps = estimator.run(stacked)

In [ ]:
significance_map = 
excess_map = 

In [ ]:
# We can plot the excess and significance maps

significance_map.plot(ax=ax1

excess_map.plot(ax=ax2

In [ ]:
## You can zoom into your region
significance_map.cutout(

## Define the model
We first define the model, a `SkyModel`, as the combination of a point source `SpatialModel` with a powerlaw `SpectralModel`:

In [ ]:
spatial_model = GaussianSpatialModel(
spectral_model = PowerLawSpectralModel(

sky_model = 

Now, we define a global `~gammapy.modeling.model.FoVBackgroundModel` in order to finely adjust the level of residual CR backgroud. This should **not** be forgotten.

In [ ]:
bkg_model = 

Now we assign these models to our reduced dataset:

In [ ]:
stacked.models = 

## Fit the model

The `~gammapy.modeling.Fit` class is orchestrating the fit, connecting the `stats` method of the dataset to the minimizer. By default, it uses `iminuit`.

In [ ]:
%%time
fit = 
result = 

Check the result of the fit

And inspect the residuals

In [ ]:
stacked.plot_residuals(

## Plot the fitted spectrum

### Making a butterfly plot 

The `SpectralModel` component can be used to produce a, so-called, butterfly plot showing the envelope of the model taking into account parameter uncertainties:

In [ ]:
spec = 
spec.plot(
ax = spec.plot_error(

### Computing flux points

We can now compute some flux points using the `~gammapy.estimators.FluxPointsEstimator`. 

In [ ]:
fpe = FluxPointsEstimator(

In [ ]:
%%time
flux_points = fpe.run(

### Inspect the results

In [ ]:
flux_points.to_table(
flux_points.plot(